# BAYESIAN NETWORK

In [ ]:
import torch
import networkx as nx
import matplotlib.pyplot as plt

class BayesianNetwork:
    def __init__(self, edges, device="cpu"):
        """
        Bayesian Network with GPU support.

        :param edges: List of (parent, child) edges representing the DAG.
        :param device: 'cpu' or 'cuda' for GPU computation.
        """
        self.device = torch.device(device)
        self.graph = nx.DiGraph()
        self.graph.add_edges_from(edges)
        self.nodes = list(self.graph.nodes())
        self.parents = {node: list(self.graph.predecessors(node)) for node in self.nodes}
        self.cpts = {}  # Conditional Probability Tables (stored as tensors)

    def visualize(self):
        """Visualizes the Bayesian Network structure."""
        plt.figure(figsize=(8, 6))
        nx.draw(self.graph, with_labels=True, node_size=3000, node_color="lightblue", edge_color="gray", font_size=12)
        plt.title("Bayesian Network Structure")
        plt.show()


In [ ]:
import torch

class OnlineBayesianEstimator:
    def __init__(self, model, alpha=1):
        """
        Online Bayesian Parameter Estimation using PyTorch (GPU accelerated).
        
        :param model: BayesianNetwork object.
        :param alpha: Laplace smoothing parameter.
        """
        self.model = model
        self.alpha = alpha  # Laplace smoothing factor
        self.counts = {}  # Stores accumulated counts for incremental learning

    def update_counts(self, batch_data):
        """
        Updates counts for each node based on new batch data.
        
        :param batch_data: Pandas DataFrame containing the new batch.
        """
        for node in self.model.nodes:
            parents = self.model.parents[node]
            if parents:
                grouped_data = batch_data.pivot_table(index=parents, columns=node, aggfunc='size', fill_value=0)
            else:
                grouped_data = batch_data[node].value_counts().to_frame().T

            if node not in self.counts:
                self.counts[node] = grouped_data
            else:
                self.counts[node] += grouped_data  # Accumulate counts across batches

    def estimate_cpds(self):
        """
        Recomputes CPDs based on updated counts.
        """
        cpts = {}
        for node, count_matrix in self.counts.items():
            smoothed_counts = count_matrix + self.alpha
            cpt_tensor = torch.tensor(smoothed_counts.div(smoothed_counts.sum(axis=1), axis=0).values,
                                      dtype=torch.float32, device=self.model.device)
            cpts[node] = cpt_tensor

        self.model.cpts = cpts  # Update model CPDs


In [ ]:
class BayesianInference:
    def __init__(self, model):
        """
        Inference in a Bayesian Network using PyTorch (GPU Accelerated).
        
        :param model: BayesianNetwork object.
        """
        self.model = model

    def compute_posterior(self, evidence):
        """
        Computes P(Label | Features) on GPU.

        :param evidence: Dictionary of observed values {feature_name: value}
        :return: Dictionary {label_value: probability}
        """
        labels = list(range(self.model.cpts['label'].shape[1]))  # Get possible label values
        posterior_probs = {}

        for label_idx in labels:
            prob = self.model.cpts['label'][0, label_idx].item()  # Extract single probability from tensor

            for feature, value in evidence.items():
                if feature in self.model.cpts:
                    feature_cpt = self.model.cpts[feature]
                    if value in range(feature_cpt.shape[0]):  # Ensure value is within valid range
                        prob *= feature_cpt[value, min(label_idx, feature_cpt.shape[1] - 1)].item()
                    else:
                        prob *= 1e-6  # Small probability for unseen values

            posterior_probs[label_idx] = prob

        # Normalize probabilities (sum-to-1 constraint)
        total = sum(posterior_probs.values())
        if total > 0:
            for label_idx in posterior_probs:
                posterior_probs[label_idx] /= total

        return posterior_probs

In [ ]:
def predict(bn_model, test_data):
    """
    Performs batch classification using Bayesian Network inference (GPU Accelerated).

    :param bn_model: Trained BayesianNetwork object.
    :param test_data: Pandas DataFrame of test instances.
    :return: Predicted labels (PyTorch tensor).
    """
    inference = BayesianInference(bn_model)
    predictions = []

    for _, row in test_data.iterrows():
        evidence = row.to_dict()
        del evidence['label']  # Remove true label (to predict it)
        posterior_probs = inference.compute_posterior(evidence)
        predicted_label = max(posterior_probs, key=posterior_probs.get)  # Argmax P(L | F)
        predictions.append(predicted_label)

    return torch.tensor(predictions, dtype=torch.int64, device=bn_model.device)


In [ ]:
import medmnist
from medmnist import PathMNIST
import torch
import torchvision.transforms as transforms
from sklearn.decomposition import PCA
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np
import pandas as pd

transform = transforms.Compose([transforms.ToTensor()])
train_dataset = PathMNIST(split='train', download=True, transform=transform)
val_dataset = PathMNIST(split='val', download=True, transform=transform)
test_dataset = PathMNIST(split='test', download=True, transform=transform)

# Convert dataset into a structured list
def preprocess_dataset(dataset, n_components=20, bins=5):
    image_list = []
    label_list = []

    for img_tensor, label_array in dataset:
        # Convert torch.Tensor to numpy.ndarray and flatten
        img_np = img_tensor.numpy().reshape(-1)  # Shape: (3*28*28,)
        
        # Extract label from numpy array
        label = label_array[0]  # Convert from [0] to scalar
        
        image_list.append(img_np)
        label_list.append(label)
    
    # Convert list to numpy array
    images_np = np.array(image_list)
    labels_np = np.array(label_list)

    # Apply PCA to reduce dimensions
    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(images_np)

    # Convert PCA features and labels into DataFrame
    df = pd.DataFrame(reduced_features, columns=[f'feature_{i}' for i in range(n_components)])
    df['label'] = labels_np  # Add label column

    # Discretize features
    discretizer = KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy='uniform')
    df.iloc[:, :-1] = discretizer.fit_transform(df.iloc[:, :-1])  # Only discretize feature columns

    # Convert to integers (Fixes float output from KBinsDiscretizer)
    df = df.astype(int)

    return df

# Apply preprocessing to training, validation, and test datasets
train_df = preprocess_dataset(train_dataset)
val_df = preprocess_dataset(val_dataset)
test_df = preprocess_dataset(test_dataset)

print(train_df.shape)
print(train_df.describe())
print(train_df.head())

In [12]:
def online_train_and_validate(train_df, val_df, batch_size=5000, model_path="best_bayesian_network.pkl", device="cuda"):
    """
    Trains a Bayesian Network using online learning (batch updates).
    
    Saves the best model based on validation F1-score.
    
    :param train_df: Training dataset (Pandas DataFrame).
    :param val_df: Validation dataset (Pandas DataFrame).
    :param batch_size: Number of samples per batch.
    :param model_path: Path to save the best model.
    :param device: "cuda" for GPU or "cpu".
    """
    # Initialize Bayesian Network structure
    edges = [(f'feature_{i}', 'label') for i in range(20)]
    bn_model = BayesianNetwork(edges, device=device)

    # Initialize Online Bayesian Estimator
    estimator = OnlineBayesianEstimator(bn_model, alpha=1)

    # Shuffle dataset before splitting into batches
    train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
    num_batches = len(train_df) // batch_size + 1  # Number of batches
    batches = np.array_split(train_df, num_batches)

    # Online Training: Process batches sequentially
    for i, batch in enumerate(batches):
        print(f"Training on batch {i+1}/{num_batches} with {len(batch)} samples...")
        estimator.update_counts(batch)  # Incrementally update counts

    # Estimate CPDs after accumulating counts
    estimator.estimate_cpds()
    print("✅ Finished online training.")

    # Run inference on validation set
    val_predictions = predict(bn_model, val_df).cpu().numpy()  # Move back to CPU for evaluation

    # Compute metrics
    y_true = val_df['label'].values
    y_pred = val_predictions

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    print(f"Validation Metrics:")
    print(f" - Accuracy:  {accuracy * 100:.2f}%")
    print(f" - Precision: {precision:.4f}")
    print(f" - Recall:    {recall:.4f}")
    print(f" - F1 Score:  {f1:.4f}")

    # Save the best model based on F1-score
    best_model = {"model": bn_model, "f1_score": f1}
    with open(model_path, "wb") as f:
        pickle.dump(best_model, f)

    print(f"✅ Best model saved at: {model_path}")

    return bn_model


In [13]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import pickle


# Define BayesianNetwork and OnlineBayesianEstimator classes (same as before)

# Helper function for hyperparameter tuning
def hyperparameter_tuning(train_df, val_df, device="cuda", n_splits=5, alpha_values=[0.1, 1, 10], batch_sizes=[1000, 5000, 10000]):
    """
    Perform hyperparameter tuning with cross-validation.

    :param train_df: Training dataset (Pandas DataFrame).
    :param val_df: Validation dataset (Pandas DataFrame).
    :param device: Device to run the model on ('cpu' or 'cuda').
    :param n_splits: Number of splits for cross-validation.
    :param alpha_values: List of alpha values to test.
    :param batch_sizes: List of batch sizes to test.
    :return: Best hyperparameters and model.
    """
    best_f1 = -1
    best_params = None
    best_model = None
    
    # K-Fold Cross-Validation
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    for alpha in alpha_values:
        for batch_size in batch_sizes:
            print(f"Testing alpha={alpha}, batch_size={batch_size}...")
            
            fold_f1_scores = []
            
            for train_index, val_index in kf.split(train_df):
                train_fold = train_df.iloc[train_index]
                val_fold = train_df.iloc[val_index]
                
                # Initialize Bayesian Network structure
                edges = [(f'feature_{i}', 'label') for i in range(20)]
                bn_model = BayesianNetwork(edges, device=device)

                # Initialize Online Bayesian Estimator
                estimator = OnlineBayesianEstimator(bn_model, alpha=alpha)

                # Shuffle dataset before splitting into batches
                train_fold = train_fold.sample(frac=1, random_state=42).reset_index(drop=True)
                num_batches = len(train_fold) // batch_size + 1  # Number of batches
                batches = np.array_split(train_fold, num_batches)

                # Online Training: Process batches sequentially
                for i, batch in enumerate(batches):
                    estimator.update_counts(batch)  # Incrementally update counts
                
                # Estimate CPDs after accumulating counts
                estimator.estimate_cpds()

                # Run inference on validation set
                val_predictions = predict(bn_model, val_fold).cpu().numpy()  # Move back to CPU for evaluation

                # Compute metrics
                y_true = val_fold['label'].values
                y_pred = val_predictions

                f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
                fold_f1_scores.append(f1)
            
            avg_f1 = np.mean(fold_f1_scores)
            print(f"Average F1 score for alpha={alpha}, batch_size={batch_size}: {avg_f1:.4f}")

            # Update best model if necessary
            if avg_f1 > best_f1:
                best_f1 = avg_f1
                best_params = {'alpha': alpha, 'batch_size': batch_size}
                best_model = bn_model
    
    print(f"Best hyperparameters found: {best_params}")
    return best_model, best_params

In [14]:
import warnings
warnings.filterwarnings('ignore')

best_model, best_params = hyperparameter_tuning(train_df, val_df, device="cuda", n_splits=5)

Testing alpha=0.1, batch_size=1000...
Average F1 score for alpha=0.1, batch_size=1000: 0.0209
Testing alpha=0.1, batch_size=5000...
Average F1 score for alpha=0.1, batch_size=5000: 0.0209
Testing alpha=0.1, batch_size=10000...
Average F1 score for alpha=0.1, batch_size=10000: 0.0209
Testing alpha=1, batch_size=1000...
Average F1 score for alpha=1, batch_size=1000: 0.0209
Testing alpha=1, batch_size=5000...
Average F1 score for alpha=1, batch_size=5000: 0.0209
Testing alpha=1, batch_size=10000...
Average F1 score for alpha=1, batch_size=10000: 0.0209
Testing alpha=10, batch_size=1000...
Average F1 score for alpha=10, batch_size=1000: 0.0209
Testing alpha=10, batch_size=5000...
Average F1 score for alpha=10, batch_size=5000: 0.0209
Testing alpha=10, batch_size=10000...
Average F1 score for alpha=10, batch_size=10000: 0.0209
Best hyperparameters found: {'alpha': 0.1, 'batch_size': 1000}


In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle

# Train with mini-batch updates
best_model = online_train_and_validate(train_df, val_df, batch_size=5000, model_path="best_bayesian_network.pkl", device="cuda")

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Training on batch 1/18 with 5000 samples...
Training on batch 2/18 with 5000 samples...
Training on batch 3/18 with 5000 samples...
Training on batch 4/18 with 5000 samples...
Training on batch 5/18 with 5000 samples...
Training on batch 6/18 with 5000 samples...
Training on batch 7/18 with 5000 samples...
Training on batch 8/18 with 5000 samples...
Training on batch 9/18 with 5000 samples...
Training on batch 10/18 with 5000 samples...
Training on batch 11/18 with 5000 samples...
Training on batch 12/18 with 5000 samples...
Training on batch 13/18 with 5000 samples...
Training on batch 14/18 with 5000 samples...
Training on batch 15/18 with 4999 samples...
Training on batch 16/18 with 4999 samples...
Training on batch 17/18 with 4999 samples...
Training on batch 18/18 with 4999 samples...
✅ Finished online training.
Validation Metrics:
 - Accuracy:  10.41%
 - Precision: 0.0116
 - Recall:    0.1111
 - F1 Score:  0.0209
✅ Best model saved at: best_bayesian_network.pkl


# AUGMENTED NAIVE BAYES (ANB)

In [10]:
import numpy as np
import networkx as nx
from sklearn.feature_selection import mutual_info_classif

def learn_anb_structure(train_df):
    """
    Learns an Augmented Naïve Bayes (ANB) structure using Mutual Information.

    :param train_df: Training dataset as Pandas DataFrame.
    :return: List of edges representing the Bayesian Network.
    """
    features = train_df.columns[:-1]  # Exclude label
    label = 'label'
    
    # Compute Mutual Information between each feature and label
    mi_scores = mutual_info_classif(train_df[features], train_df[label])

    # Create a complete graph and assign MI scores as weights
    G = nx.Graph()
    for i, feature in enumerate(features):
        G.add_edge(label, feature, weight=mi_scores[i])  # Connect features to label

    # Compute Mutual Information between features
    for i, f1 in enumerate(features):
        for j, f2 in enumerate(features):
            if i < j:  # Avoid duplicate edges
                mi = mutual_info_classif(train_df[[f1]], train_df[f2])[0]
                G.add_edge(f1, f2, weight=mi)

    # Use **Maximum Spanning Tree (MST)** to find best dependencies
    mst = nx.maximum_spanning_tree(G)

    # Convert MST to a Directed Graph (DAG)
    DAG = nx.DiGraph()
    for edge in mst.edges(data=True):
        parent, child, _ = edge
        DAG.add_edge(parent, child)

    return list(DAG.edges)


In [11]:
import os
import pickle
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

def train_anb(train_df, test_df, val_df, model_path="best_anb.pkl"):
    """
    Trains an Augmented Naïve Bayes (ANB) model with learned feature dependencies.
    
    Saves the best model based on validation F1-score.
    
    :param train_df: Training dataset (Pandas DataFrame).
    :param test_df: Testing dataset (Pandas DataFrame).
    :param val_df: Validation dataset (Pandas DataFrame).
    :param model_path: Path to save the best model.
    """
    print("Starting Augmented Naïve Bayes (ANB) model training...")

    # Learn feature dependencies
    edges = learn_anb_structure(train_df)
    print(f"Learned ANB Structure: {edges}")

    # Extract features and labels
    X_train, y_train = train_df.iloc[:, :-1], train_df.iloc[:, -1]
    X_test, y_test = test_df.iloc[:, :-1], test_df.iloc[:, -1]
    X_val, y_val = val_df.iloc[:, :-1], val_df.iloc[:, -1]

    # Train ANB Model with Hyperparameter Tuning
    print("Training Augmented Naïve Bayes model...")
    param_grid = {
        'alpha': [0.01, 0.1, 0.5, 1.0, 2.0],
        'fit_prior': [True, False]
    }

    anb_model = MultinomialNB()
    grid_search = GridSearchCV(anb_model, param_grid, cv=5, scoring='f1_weighted')
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    print(f"Best hyperparameters: {grid_search.best_params_}")

    # Evaluate on test data
    y_test_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(classification_report(y_test, y_test_pred))

    # Evaluate on validation data
    y_val_pred = best_model.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, average='macro', zero_division=0)
    recall = recall_score(y_val, y_val_pred, average='macro', zero_division=0)
    f1 = f1_score(y_val, y_val_pred, average='macro', zero_division=0)

    print("Validation Metrics:")
    print(f" - Accuracy:  {val_accuracy * 100:.2f}%")
    print(f" - Precision: {precision:.4f}")
    print(f" - Recall:    {recall:.4f}")
    print(f" - F1 Score:  {f1:.4f}")

    # Save the best model
    model_data = {
        'model': best_model,
        'feature_dependencies': edges,  # Save the learned feature dependencies
        'metrics': {
            'test_accuracy': test_accuracy,
            'validation_accuracy': val_accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'best_params': grid_search.best_params_
        }
    }

    save_path = os.path.join("models", model_path)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)  # Ensure directory exists
    with open(save_path, 'wb') as f:
        pickle.dump(model_data, f)

    print(f"Augmented Naïve Bayes model saved at: {save_path}")

    return model_data


In [12]:
# Train ANB model
trained_anb_model = train_anb(train_df, test_df, val_df, model_path="best_anb.pkl")


Starting Augmented Naïve Bayes (ANB) model training...
Learned ANB Structure: [('label', 'feature_1'), ('label', 'feature_0'), ('label', 'feature_9'), ('label', 'feature_13'), ('label', 'feature_14'), ('label', 'feature_12'), ('feature_2', 'feature_7'), ('feature_2', 'feature_4'), ('feature_2', 'feature_3'), ('feature_2', 'feature_10'), ('feature_4', 'feature_12'), ('feature_4', 'feature_16'), ('feature_3', 'feature_8'), ('feature_10', 'feature_15'), ('feature_10', 'feature_19'), ('feature_8', 'feature_17'), ('feature_5', 'feature_6'), ('feature_5', 'feature_10'), ('feature_6', 'feature_11'), ('feature_17', 'feature_18')]
Training Augmented Naïve Bayes model...
Best hyperparameters: {'alpha': 0.5, 'fit_prior': False}
Test Accuracy: 0.4078
              precision    recall  f1-score   support

           0       0.96      0.83      0.89      1338
           1       0.48      0.97      0.64       847
           2       0.16      0.02      0.04       339
           3       0.00      0.00 

# HIDDEN MARKOV MODEL (HMM)

In [15]:
import os
import pickle
import numpy as np
import pandas as pd
from hmmlearn import hmm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

class HiddenMarkovModelTrainer:
    def __init__(self):
        self.hmm_models = None  # Store trained HMM models

    def train_hmm(self, train_df, test_df, val_df, n_components_range=[2, 3, 4], random_state=42, model_path="hmm_models.pkl"):
        """
        Trains a Hidden Markov Model (HMM) for each class in the dataset with cross-validation and hyperparameter tuning.

        Args:
            train_df: Training dataset (Pandas DataFrame).
            test_df: Testing dataset (Pandas DataFrame).
            val_df: Validation dataset (Pandas DataFrame).
            n_components_range: List of possible numbers of hidden states (components) for hyperparameter tuning.
            random_state: Random seed for reproducibility.
            model_path: Path to save the trained HMM models.
            
        Returns:
            A dictionary containing the trained HMM models.
        """
        print("Starting Hidden Markov Model (HMM) training...")

        # Extract features and labels
        X_train, y_train = train_df.iloc[:, :-1], train_df.iloc[:, -1]
        X_test, y_test = test_df.iloc[:, :-1], test_df.iloc[:, -1]
        X_val, y_val = val_df.iloc[:, :-1], val_df.iloc[:, -1]

        # Normalize the feature data using StandardScaler
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        X_val_scaled = scaler.transform(X_val)

        # Perform K-fold cross-validation for hyperparameter tuning
        kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
        best_hmm_models = {}
        best_f1 = -1  # Initialize best F1 score as a very low value

        # Iterate over possible n_components for hyperparameter tuning
        for n_components in n_components_range:
            hmm_models = {}
            avg_f1_score = 0  # Average F1 score for this n_components across folds

            for train_idx, val_idx in kf.split(X_train_scaled):
                X_train_fold, X_val_fold = X_train_scaled[train_idx], X_train_scaled[val_idx]
                y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

                # Train HMM for each class
                for label in np.unique(y_train_fold):
                    X_label = X_train_fold[y_train_fold == label]

                    if len(X_label) < n_components:
                        print(f"Warning: Not enough samples ({len(X_label)}) for label {label} with {n_components} components")
                        continue

                    model = hmm.GaussianHMM(
                        n_components=n_components,
                        covariance_type="full",
                        n_iter=200,
                        random_state=random_state
                    )

                    try:
                        model.fit(X_label)  # Fit the model to the class-specific data
                        hmm_models[label] = model
                    except Exception as e:
                        print(f"Error training HMM for label {label}: {e}")

                # Evaluate on validation fold
                y_val_pred = self.predict_hmm(hmm_models, X_val_fold)
                f1 = f1_score(y_val_fold, y_val_pred, average='macro', zero_division=0)
                avg_f1_score += f1

            avg_f1_score /= kf.get_n_splits()
            print(f"Average F1 score for n_components={n_components}: {avg_f1_score:.4f}")

            # Update best model if necessary
            if avg_f1_score > best_f1:
                best_f1 = avg_f1_score
                best_hmm_models = hmm_models
                print(f"Updated best model with n_components={n_components}")

        # Save the best model
        save_path = os.path.join("models", model_path)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        with open(save_path, "wb") as f:
            pickle.dump(best_hmm_models, f)

        print(f"HMM models saved at: {save_path}")
        self.hmm_models = best_hmm_models  # Store the best models for later use
        return best_hmm_models

    def predict_hmm(self, hmm_models, X_test_scaled):
        """
        Predicts labels using trained HMM models.

        Args:
            hmm_models: Dictionary of trained HMM models.
            X_test_scaled: Scaled feature matrix.

        Returns:
            List of predicted labels.
        """
        X_test_scaled = np.array(X_test_scaled)
        predictions = []

        for i in range(X_test_scaled.shape[0]):
            x = X_test_scaled[i].reshape(1, -1)  # Correct reshaping for single sample
            max_log_prob = float('-inf')
            best_label = None

            for label, model in hmm_models.items():
                try:
                    log_prob = model.score(x)
                    if log_prob > max_log_prob:
                        max_log_prob = log_prob
                        best_label = label
                except:
                    continue

            predictions.append(best_label if best_label is not None else -1)

        return np.array(predictions)

    def evaluate_hmm(self, hmm_models, X_test_scaled, y_test, X_val_scaled, y_val):
        """
        Evaluates the HMM model on test and validation sets.

        Args:
            hmm_models: Dictionary of trained HMM models.
            X_test_scaled: Scaled test features.
            y_test: Test labels.
            X_val_scaled: Scaled validation features.
            y_val: Validation labels.
        
        Returns:
            A dictionary of evaluation metrics.
        """
        # Predict on test set
        y_test_pred = self.predict_hmm(hmm_models, X_test_scaled)
        test_accuracy = accuracy_score(y_test, y_test_pred)
        print(f"Test Accuracy: {test_accuracy:.4f}")
        print(classification_report(y_test, y_test_pred))

        # Predict on validation set
        y_val_pred = self.predict_hmm(hmm_models, X_val_scaled)
        val_accuracy = accuracy_score(y_val, y_val_pred)
        precision = precision_score(y_val, y_val_pred, average='macro', zero_division=0)
        recall = recall_score(y_val, y_val_pred, average='macro', zero_division=0)
        f1 = f1_score(y_val, y_val_pred, average='macro', zero_division=0)

        print("Validation Metrics:")
        print(f" - Accuracy:  {val_accuracy * 100:.2f}%")
        print(f" - Precision: {precision:.4f}")
        print(f" - Recall:    {recall:.4f}")
        print(f" - F1 Score:  {f1:.4f}")

        return {
            'test_accuracy': test_accuracy,
            'validation_accuracy': val_accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        }

# Initialize and train the HMM model
hmm_trainer = HiddenMarkovModelTrainer()
trained_hmm_models = hmm_trainer.train_hmm(train_df, test_df, val_df)

# Load and evaluate the model
hmm_metrics = hmm_trainer.evaluate_hmm(
    trained_hmm_models, 
    test_df.iloc[:, :-1].values, test_df.iloc[:, -1].values, 
    val_df.iloc[:, :-1].values, val_df.iloc[:, -1].values
)

In [23]:
# Initialize and train the HMM model
hmm_trainer = HiddenMarkovModelTrainer()
trained_hmm_models = hmm_trainer.train_hmm(train_df, test_df, val_df)

# Load and evaluate the model
hmm_metrics = hmm_trainer.evaluate_hmm(
    trained_hmm_models, 
    test_df.iloc[:, :-1].values, test_df.iloc[:, -1].values, 
    val_df.iloc[:, :-1].values, val_df.iloc[:, -1].values
)

Starting Hidden Markov Model (HMM) training...
Successfully trained HMM for label 0 with 9366 samples
Successfully trained HMM for label 1 with 9509 samples
Successfully trained HMM for label 2 with 10360 samples
Successfully trained HMM for label 3 with 10401 samples
Successfully trained HMM for label 4 with 8006 samples
Successfully trained HMM for label 5 with 12182 samples
Successfully trained HMM for label 6 with 7886 samples
Successfully trained HMM for label 7 with 9401 samples
Successfully trained HMM for label 8 with 12885 samples
HMM models saved at: models/hmm_models.pkl
Test Accuracy: 0.1180
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1338
           1       0.12      1.00      0.21       847
           2       0.00      0.00      0.00       339
           3       0.00      0.00      0.00       634
           4       0.00      0.00      0.00      1035
           5       0.00      0.00      0.00       592
          

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Metrics:
 - Accuracy:  10.57%
 - Precision: 0.0117
 - Recall:    0.1111
 - F1 Score:  0.0212
